In [1]:
import pandas as pd
import dash
from dash import html
from dash import dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
value_all = spacex_df.groupby('Launch Site')['class'].sum()
names_all = spacex_df.groupby('Launch Site')['Launch Site'].first()
names_specific = spacex_df['class'].unique()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(
    children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
            # TASK 1: Add a dropdown list to enable Launch Site selection
            # The default select value is for ALL sites
            # dcc.Dropdown(id='site-dropdown',...)
            dcc.Dropdown(id = 'site-dropdown',
                        options=[
                     {'label': 'All Sites', 'value': 'All Sites'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                 ],
                 value='All Sites',
                 searchable=True,
                 placeholder = 'Select a Launch Site'
                 ),
            html.Br(),
            # TASK 2: Add a pie chart to show the total successful launches count for all sites
            # If a specific launch site was selected, show the Success vs. Failed counts for the site
            html.Div(dcc.Graph(id='success-pie-chart')),
            html.Br(),
              
            
            html.P("Payload range (Kg):"),
            # TASK 3: Add a slider to select payload range
            #dcc.RangeSlider(id='payload-slider',
            dcc.RangeSlider(id ='payload-slider', min = 0, max = 10000, step = 1000,
                           value = [min_payload, max_payload]),
              
              
            # TASK 4: Add a scatter chart to show the correlation between payload and launch success
            html.Div(dcc.Graph(id='success-payload-scatter-chart')),
            html.Br(),
              
            html.P("Bar chart for Booster Version Category and launh success"),
            html.Div(dcc.Graph(id = 'bar-chart'))
            ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id = 'success-pie-chart', component_property = 'figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'))

def get_pie_chart(entered_site):
    if entered_site == 'All Sites':
        fig = px.pie(spacex_df, values = value_all , names = names_all, 
                     title = 'Pie chart showing success percentage for all launch sites')
    else:
        fig = px.pie(spacex_df, 
                     values = spacex_df[spacex_df['Launch Site'] == str(entered_site)]['class'].value_counts()
                     ,names = names_specific,
                    title = f'Pie chart showing success percentage for the launch site {str(entered_site)}')
    return(fig)

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
             Input(component_id = 'site-dropdown',  component_property = 'value'),
             Input(component_id = 'payload-slider',  component_property = 'value'))


def get_scatter(site, slider_range):
    low = slider_range[0]
    high = slider_range[1]
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    if site == "All Sites":
       fig = px.scatter(spacex_df.loc[mask] , x = 'Payload Mass (kg)', y = 'class',
                color = 'Booster Version Category', hover_data = ['Launch Site'],
                title = 'Scatter plot displaying correlation between payload mass and success for all sites')
    else:
        df = spacex_df.copy()
        df = df[df['Launch Site'] == str(site)]
        fig = px.scatter(df.loc[mask],x = 'Payload Mass (kg)', y = 'class',
             color = 'Booster Version Category', hover_data = ['Launch Site'],
            title = f'Scatter plot displaying correlation between payload mass and success for {str(site)}')
    return(fig)


@app.callback( Output(component_id = 'bar-chart', component_property = 'figure'),
            Input(component_id = 'site-dropdown', component_property = 'value'))
def get_bar(site):
    df_boost = spacex_df.groupby('Booster Version Category')['class'].sum().reset_index(name = 'count')
    fig  = px.bar(df_boost, x = 'Booster Version Category', y = 'count')
    return(fig)
# Run the app
if __name__ == '__main__':
    app.run_server(debug = True)


C:\Users\shris\AppData\Local\Temp\ipykernel_8976\1130009546.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# <dl>
<dd>1. Launch Site KSC LC-39A(41.7%) has the largest successful launches</dd>
<dd>2. Launch Site KSC LC-39A(76.9%) has the largest successful launches.</dd>
<dd>3. Payload mass between 0-2k kg and 6k-10k has the lowest launch success rate .</dd>
<dd>4. F9 Booster version FT has the highest success rate for all sites.</dd>
<dd>5. CCAFS LC-40 with payload range between 4k-10k kg has 0% of successful launch!.</dd>

</dl>